# Azure AI Agent service - Bing custom search

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [ ]:
#%pip install azure-ai-projects

In [ ]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingCustomSearchTool, MessageRole
from dotenv import load_dotenv

In [ ]:
load_dotenv("azure.env")

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 10-Jun-2025 11:16:55


## 1. Grounding with Bing Custom Search

### Connect to AI Foundry

In [5]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to custom bing grounding service, the config only has ARXIV.ORG in it !!

In [13]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("BING_CUSTOM_CONNECTION_ID")

# Initialize Bing Custom Search tool with connection id and instance name
bing_custom_tool = BingCustomSearchTool(connection_id=conn_id, instance_name="first-config")

### Create agent with Bing Grounding tool

In [14]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-bing-grounding-agent",
        instructions=instructions,
        tools=bing_custom_tool.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

bing_custom_tool
bing_custom_tool.definitions


Created agent, ID: asst_DWuXDWATMqk7lDRjufogbmpH


[{'type': 'bing_custom_search', 'bing_custom_search': {'search_configurations': [{'connection_id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqbingcustomsearch002', 'instance_name': 'first-config', 'market': '', 'set_lang': '', 'count': 5, 'freshness': ''}]}}]

### Create THREAD

In [15]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_tJApoH9hs2OFuBXqxyWiN1dZ


### Example #1

In [25]:
query = "what is are the latest news on AI?"

### Create first message in the thread

In [26]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_vZZHqX3wPpIWQr4c3rl4Akaz


### Create first RUN

In [27]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


### Check status of the RUN and tool calls

In [28]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage


Step step_zenFQWPdNfkGHuQ9e7k9twbp status: completed
Step step_xNlRYqWoeE2J4j9yfrB2VPvH status: completed
  Tool calls:
    Tool Call ID: call_Hg8DisL0cYoloKbVShwCMKmp
    Type: bing_custom_search



{'prompt_tokens': 1659, 'completion_tokens': 224, 'total_tokens': 1883, 'prompt_token_details': {'cached_tokens': 0}}

### Get response

In [29]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: Here are some of the latest developments in artificial intelligence as of June 2025:

1. The 2025 AI Index Report has been released, offering a comprehensive overview of AI’s growing influence on society, economies, and global governance. This report highlights the increasing integration of AI across various sectors and the importance of understanding its impact on a global scale【8:1†source】.

2. New research discusses futuristic AI and machine learning models, including intelligent systems, and will be presented at major conferences such as the IEEE International Workshop on Advances in Artificial Intelligence and Machine Learning (AIML 2025)【8:0†source】.

3. Innovations in safe and sustainable medical AI systems are in focus, with projects like ontology-based knowledge representation for bone disease diagnosis illustrating how AI is advancing healthcare technologies【8:3†source】.

These trends reflect ongoing research and significant advancements, particularly in areas

### Example #2

In [30]:
query = "find out about the latest iPhone"

In [31]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_HhDuVNTp13lWGO2akIhNpuLN


In [32]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [33]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_5TtdU9RWgmQm8m0CshqaxWsG status: completed
Step step_Z5ttqQ1t2BVGRe7opkxIFhqp status: completed
  Tool calls:
    Tool Call ID: call_u85J2cjfTmJpZU9HnFP3gUqY
    Type: bing_custom_search



{'prompt_tokens': 2266, 'completion_tokens': 172, 'total_tokens': 2438, 'prompt_token_details': {'cached_tokens': 0}}

In [34]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: There isn't a detailed announcement or confirmed features for a brand-new iPhone released in 2025 as of now. The last widely-covered iPhone launch was Apple's September 2023 event, where they introduced the most recent iPhone model. These annual launches are significant because they tend to set smartphone trends and showcase Apple's advancements in processing power, camera technology, and AI features.

While there are ongoing research and improvements in mobile hardware (such as increases in NPU processing power and DRAM size in new iPhones), no major details or official announcements are available yet for an iPhone launched specifically in 2025【12:0†source】【12:1†source】. For real-time updates, checking Apple's official newsroom or trusted tech news sources is recommended.
URL Citation: [Bibek Paudel Preetam Joshi Puneet Anand AIMon Labs - arXiv.org](https://arxiv.org/pdf/2504.07069)
URL Citation: [ABSTRACT arXiv:2412.01380v2 [cs.LG] 3 Apr 2025](https://arxiv.org/pdf/24

### Example #3

In [35]:
query = "What is the weather informations for Mexico city? Show the report and use emojis and be funny"

In [36]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_7wgA1QOXuT39ylo3xQYTYHO0


In [37]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [38]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_SUCwRxxdAqA4HkQD3RUqMGzF status: completed
Step step_wf76w7zoqFI8gZ6FDfat0Ss6 status: completed
  Tool calls:
    Tool Call ID: call_uBUpmwftnMHIbKH1JAuuMead
    Type: bing_custom_search



{'prompt_tokens': 2558, 'completion_tokens': 238, 'total_tokens': 2796, 'prompt_token_details': {'cached_tokens': 0}}

In [39]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: Sorry, I'm unable to get the live weather data for Mexico City at the moment. But since you wanted a funny forecast with emojis, here’s a “just-for-fun” report:

🌞 Good morning, Mexico City! Today, expect a battle between Señor Sun and Señor Cloud—who will win? 🎭  
☁️ A few clouds might roll in to try and photobomb your Instagram story, but don’t worry, the 🌞 sun is still the main star!  
💨 Expect a gentle breeze, just strong enough to give your hair that accidentally-on-purpose “windblown model” look.  
🌡️ The temperature? Not too hot, not too cold—just right for tacos and a stroll in Chapultepec Park! 🌮🌳  
☂️ Pro tip: Umbrellas might not be needed, but you never know when a rogue pigeon will make you wish you had one! 🕊️☂️

For accurate, real-time info, check your favorite weather app, or let me know if you'd like me to try again!


### Post processing

In [40]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_5kZsBE4Yh1CyX9SGdt69cqfl, Name: Agent34, Model: gpt-4.1, Instructions: 
Agent ID: asst_DWuXDWATMqk7lDRjufogbmpH, Name: my-bing-grounding-agent, Model: gpt-4.1, Instructions: You are an AI Agent that can do some web search using Bing
Agent ID: asst_rhSoPMLILRvTXAn5FnIWXCu4, Name: my-bing-grounding-agent, Model: gpt-4.1, Instructions: You are an AI Agent that can do some web search using Bing
Agent ID: asst_sO5reTYCQVM5QUQ3M7SaL5IB, Name: my-bing-grounding-agent, Model: gpt-4.1, Instructions: You are an AI Agent that can do some web search using Bing


In [42]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [43]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")